In [1]:
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from PIL import Image

# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True)

# Load pre-trained FaceNet model
resnet = InceptionResnetV1(pretrained='casia-webface').eval()

# Load an image containing faces
path = os.path.join("..","in","newspapers")

for page in newspaper:
    img = Image.open('path_to_image.jpg')

# Detect faces in the image
boxes, _ = mtcnn.detect(img)

100.0%


KeyboardInterrupt: 